In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline


/home/penglab/anaconda3/lib/python3.7/site-packages/neuro_morpho_toolbox/
Loading CCF Atlas data...
Loading time: 1.06
Loading CCF brain structure data...
Loading time: 16.25
Loading...
number of files under swc_path: 1002
100 loaded: 2.8s
Invalid number of soma found: 0
QC failed: AA0114.swc
Invalid number of soma found: 0
QC failed: AA0115.swc
200 loaded: 2.1s
300 loaded: 2.5s
400 loaded: 2.6s
Invalid number of soma found: 0
QC failed: AA0472.swc
500 loaded: 2.5s
Invalid number of soma found: 0
QC failed: AA0576.swc
Invalid number of soma found: 0
QC failed: AA0585.swc
Invalid number of soma found: 0
QC failed: AA0589.swc
600 loaded: 2.3s
Invalid number of soma found: 0
QC failed: AA0639.swc
Invalid number of soma found: 0
QC failed: AA0670.swc
Invalid number of soma found: 0
QC failed: AA0672.swc
700 loaded: 2.2s
Invalid number of soma found: 0
QC failed: AA0754.swc
Invalid number of soma found: 0
QC failed: AA0763.swc
800 loaded: 2.7s
900 loaded: 2.1s
QC failed: AA0952.swc
QC faile

/home/penglab/anaconda3/lib/python3.7/site-packages/neuro_morpho_toolbox/neuron_features.py:337: RuntimeWarning:

divide by zero encountered in true_divide

/home/penglab/anaconda3/lib/python3.7/site-packages/neuro_morpho_toolbox/neuron_features.py:337: RuntimeWarning:

invalid value encountered in true_divide



In [43]:
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
 
    return list(duplicateColumnNames)

### function readVaa3dFeature(addr, nameF)
* **addr** is the location of that plain txt generated by Vaa3D
* **nameF** is the name of that feature

from IPython.display import display

def readVaa3DFeature(addr, nameF):
    df = pd.read_csv(addr, sep=r'\t', engine='python').transpose()
    df.rename(columns = df.loc['ID',:], inplace=True)
    df.drop(index='ID', inplace = True) 
    print('Drop duplicate columns '+ str(getDuplicateColumns(df)))
    df.drop(columns = getDuplicateColumns(df),inplace = True)
    print('Loading ' + str(nameF) +' features successfully, the shape of that dataframe is '+ str(df.shape))
    col_mask = df.isnull().any(axis=0) 
    row_mask = df.isnull().any(axis=1) 
    if not df.loc[row_mask,col_mask]:
        print('NAN value exists for the following feature:')
        display(df.loc[row_mask,col_mask])
    return df

In [125]:
from IPython.display import display

def readVaa3DFeature(addr, nameF):
    print('Loading ' + str(nameF) +' features') 
    df = pd.read_csv(addr, sep=r'\t', header=[0], index_col=[0], delimiter="\t").transpose()
    df.rename(columns={'Number of Bifurcatons':'Number of Bifurcations'}, inplace=True)
    use_cols = ['Number of Stems', 
            'Overall Width', 'Overall Height', 'Overall Depth', 
            'Total Length', 
            'Max Euclidean Distance', 'Max Path Distance', 
            'Number of Bifurcations', 'Number of Branches', 'Number of Tips',
            'Max Branch Order','Average Contraction', 'Average Fragmentation',
            'Average Bifurcation Angle Local', 'Average Bifurcation Angle Remote', 
            'Hausdorff Dimension'
           ]
    df = df[use_cols]
    feature_name = use_cols
    if nameF == 'axon':
        new_feature_name = ['A_'+i.replace(' ', '_') for i in use_cols]
    if nameF == 'proximal axon':
        new_feature_name = ['AL_'+i.replace(' ', '_') for i in use_cols]
    if nameF == 'dendrite':
        new_feature_name = ['D_'+i.replace(' ', '_') for i in use_cols]
    
    df.rename(columns=dict(zip(feature_name, new_feature_name)), inplace=True)

    col_mask = df.isnull().any(axis=0) 
    row_mask = df.isnull().any(axis=1) 
    if not (df.loc[row_mask,col_mask]).shape == (0,0):
        print('NAN value exists for the following feature:')
        display(df.loc[row_mask,col_mask])
        df.drop(index = df.loc[row_mask,col_mask].index, inplace = True)
        print('Related .swc files were removed')
    print('Loading successfully, the shape of that dataframe is '+ str(df.shape))
    return df

## Axon features

In [126]:
axonFea = readVaa3DFeature('/home/penglab/FeaCal/Jmorpho_features/axon.features/temp', 'axon')

Loading axon features
Loading successfully, the shape of that dataframe is (991, 16)


## Dendrite features

In [127]:
dendriteFea = readVaa3DFeature('/home/penglab/FeaCal/Jmorpho_features/dendrite.features/temp', 'dendrite')


Loading dendrite features
NAN value exists for the following feature:


ID,D_Average_Contraction,D_Average_Fragmentation,D_Average_Bifurcation_Angle_Local,D_Average_Bifurcation_Angle_Remote
AA0309,0.814347,58.0,NaN,NaN
AA0411,NaN,NaN,NaN,NaN


Related .swc files were removed
Loading successfully, the shape of that dataframe is (989, 16)


## Proximal axon features

In [128]:
proxi_axonFea = readVaa3DFeature('/home/penglab/FeaCal/Jmorpho_features/proximal_axon.features/temp', 'proximal axon')

Loading proximal axon features
NAN value exists for the following feature:


ID,AL_Average_Bifurcation_Angle_Local,AL_Average_Bifurcation_Angle_Remote
AA0018,NaN,NaN
AA0023,NaN,NaN
AA0029,NaN,NaN
AA0038,NaN,NaN
AA0047,NaN,NaN
AA0048,NaN,NaN
AA0049,NaN,NaN
AA0050,NaN,NaN
AA0051,NaN,NaN
AA0052,NaN,NaN


Related .swc files were removed
Loading successfully, the shape of that dataframe is (720, 16)


# Deal with data 

In [168]:

def scale(df, log=True):
    scaled_data = np.array(df) / np.sum(df, axis=1).values.reshape(-1,1)
    scaled_data[np.isnan(scaled_data)]=0
    return scaled_data


### Concated Axon

In [177]:


lm_axon = nmt.features("L-measure of axon")
lm_axon.add_raw_data(pd.concat([axonFea, proxi_axonFea], axis=1,sort=False))


lm_axon_df = lm_axon.raw_data.copy()
use_cols = ['A_Overall_Width', 
            'A_Overall_Height', 
            'A_Overall_Depth', 
            'A_Total_Length', 
            'A_Max_Euclidean_Distance', 
            'A_Max_Path_Distance', 
            'A_Number_of_Branches', 
#             'Max Branch Order',
#             'Average Contraction', 
#             'Average Fragmentation',
#             'Average Bifurcation Angle Local', 
#             'Average Bifurcation Angle Remote', 
#             'Hausdorff Dimension',
            'AL_Total_Length',
            'AL_Number_of_Branches'
           ]
lm_axon_df = lm_axon_df[use_cols]

col_mask = lm_axon_df.isnull().any(axis=0) 
row_mask = lm_axon_df.isnull().any(axis=1) 
if not (lm_axon_df.loc[row_mask,col_mask]).shape == (0,0):
    print('NAN value exists for the following feature:')
    display(lm_axon_df.loc[row_mask,col_mask])
    lm_axon_df.drop(index = lm_axon_df.loc[row_mask,col_mask].index, inplace = True)
    print('Related feature samples were removed')
print('Loading successfully, the shape of that dataframe is '+ str(lm_axon_df.shape))

lm_axon_df_scale = pd.DataFrame(scale(lm_axon_df), 
                                index=lm_axon_df.index, 
                                columns=lm_axon_df.columns)

Number of input neurons: 991
Number of input features: 32
NAN value exists for the following feature:


ID,AL_Total_Length,AL_Number_of_Branches
AA0018,NaN,NaN
AA0023,NaN,NaN
AA0029,NaN,NaN
AA0038,NaN,NaN
AA0047,NaN,NaN
AA0048,NaN,NaN
AA0049,NaN,NaN
AA0050,NaN,NaN
AA0051,NaN,NaN
AA0052,NaN,NaN


Related feature samples were removed
Loading successfully, the shape of that dataframe is (720, 9)


In [179]:
lm_axon_df_scale.to_excel('/home/penglab/FeaCal/Jmorpho_features/lm_axon_df_scale.xlsx')

### Dendrite features

In [181]:
lm_dendrite = nmt.features("L-measure of dendrite")
lm_dendrite.add_raw_data(dendriteFea)



lm_dendrite_df = lm_dendrite.raw_data.copy().loc[:,:]
use_cols = [
    'D_Number_of_Stems', 
    'D_Overall_Width', 
    'D_Overall_Height', 
    'D_Overall_Depth', 
    'D_Total_Length',
    'D_Max_Euclidean_Distance', 
    'D_Max_Path_Distance', 
    'D_Number_of_Branches', 
    'D_Max_Branch_Order',
]
lm_dendrite_df = lm_dendrite_df[use_cols]
lm_dendrite_df["D_Depth_Width-Ratio"] = lm_dendrite_df["D_Overall_Depth"] / lm_dendrite_df["D_Overall_Width"]


col_mask = lm_dendrite_df.isnull().any(axis=0) 
row_mask = lm_dendrite_df.isnull().any(axis=1) 
if not (lm_dendrite_df.loc[row_mask,col_mask]).shape == (0,0):
    print('NAN value exists for the following feature:')
    display(lm_dendrite_df.loc[row_mask,col_mask])
    lm_dendrite_df.drop(index = lm_dendrite_df.loc[row_mask,col_mask].index, inplace = True)
    print('Related feature samples were removed')
print('Loading successfully, the shape of that dataframe is '+ str(lm_dendrite_df.shape))


lm_dendrite_df_scale = pd.DataFrame(scale(lm_dendrite_df), 
                                    index=lm_dendrite_df.index, 
                                    columns=lm_dendrite_df.columns
                                   )


Number of input neurons: 989
Number of input features: 16
Loading successfully, the shape of that dataframe is (989, 10)


In [182]:
lm_dendrite_df_scale.to_excel('/home/penglab/FeaCal/Jmorpho_features/lm_dendrite_df_scale.xlsx')

# Calculate co-clustering matrix


## Using axon morphology as features
SETTING CLUSTER NUMBER FROM 8 TO 40

### Hierarchy Clustering
For Hierarchy method
* the most suitable parameter is {'L_method': 'weighted', 'L_metric': 'mahalanobis', 'criterionH': 'distance', 'depth': 2, 'R': None, 't': 0.9, 'optimal_ordering': False}
* the ARI is 0.08785414289406115
* The setting cluster number's limit is satisfied, the final number of cluster is 8

### Kmeans Clustering
For Kmeans method
* the most suitable parameter is {'n_clusters': 8, 'init': 'k-means++', 'n_init': 21, 'max_iter': 300, 'tol': 0.0001, 'precompute_distances': False, 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None, 'algorithm': 'auto'}
* the ARI is 0.08343509698163794
* The setting cluster number's limit is satisfied, the final number of cluster is 8

### DBSCAN Clustering
For DBSCAN method
* the most suitable parameter is {'eps': 0.31, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 'leaf_size': 30, 'p': None, 'n_jobs': None}
* the ARI is 0.10187925011450155
* The setting cluster number's limit is satisfied, the final number of cluster is 8

### HDBSCAN Clustering
For HDBSCAN method, 
* the most suitable parameter is {'min_cluster_size': 5, 'metric': 'manhattan', 'alpha': 0.8, 'min_samples': 3, 'p': 2, 'algorithm': 'best', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 'prediction_data': False, 'match_reference_implementation': False}
* the ARI is 0.05177167199451809
* The setting cluster number's limit is satisfied, the final number of cluster is 39

### SNN Clustering
For SNN_community method
* the most suitable parameter is {'knn': 5, 'metric': 'minkowski', 'method': 'FastGreedy'}
* the ARI is 0.06673692576743263
* The setting cluster number's limit is satisfied, the final number of cluster is 23

In [188]:
par_hier =  {'L_method': 'weighted', 'L_metric': 'mahalanobis', 'criterionH': 'distance', 'depth': 2, 'R': None, 
             't': 0.9, 'optimal_ordering': False}
par_kmeans = {'n_clusters': 8, 'init': 'k-means++', 'n_init': 21, 'max_iter': 300, 'tol': 0.0001, 
              'precompute_distances': False, 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None,
              'algorithm': 'auto'}

par_dbscan = {'eps': 0.31, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 
              'leaf_size': 30, 'p': None, 'n_jobs': None}

par_hdbscan = {'min_cluster_size': 5, 'metric': 'manhattan', 'alpha': 0.8, 'min_samples': 3, 'p': 2, 'algorithm':
               'best', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 
               'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 
               'prediction_data': False, 'match_reference_implementation': False}

par_snn = {'knn':5,'metric':'minkowski','method':'FastGreedy'}


In [189]:
def get_clusters(inputUMAP,method='SNN_community',karg_dict={'knn':5, 'metric':'minkowski','method':'FastGreedy'}):
    methods_allowed = ['SNN_community', 'Hierarchy', 'Kmeans', 'DBSCAN', 'HDBSCAN']
    assert method in methods_allowed, "Please set 'method' as one of the following: 'SNN_community', 'Hierarchy', 'Kmeans', 'DBSCAN', 'HDBSCAN'"
    selectedUMAP = inputUMAP.copy()
    if method=='SNN_community':
        #print('Result of SNN_community')
        if 'knn' in karg_dict.keys():
            knn = karg_dict['knn']
        else:
            knn = 5
        if 'metric' in karg_dict.keys():
            metric = karg_dict['metric']
        else:
            metric = 'minkowski'
        if 'method' in karg_dict.keys():
            community_method = karg_dict['method']
        else:
            community_method = 'FastGreedy'
        cur_clusters = nmt.get_clusters_SNN_community(selectedUMAP, knn=knn, metric=metric, method=community_method)
        

    #karg_dict={'L_method':'single','L_metric':'euclidean'.'t':0.9,'criterionH':'inconsistent', depth=2, R=None, monocrit=None}
    if method =='Hierarchy':
        #print('Result of Hierarchy CLustering')
        cur_clusters = nmt.get_clusters_Hierarchy_clustering(selectedUMAP, karg_dict)


    if method =='Kmeans':
        #print('Result of Kmeans')
        cur_clusters = nmt.get_clusters_kmeans_clustering(selectedUMAP, karg_dict)

    if method =='DBSCAN':
        #print('Result of DBSCAN')
        cur_clusters = nmt.get_clusters_dbscan_clustering(selectedUMAP, karg_dict)

    if method =='HDBSCAN':
        #print('Result of HDBSCAN')
        cur_clusters = nmt.get_clusters_hdbscan_clustering(selectedUMAP, karg_dict)
    selectedUMAP.loc[:,'Cluster'] = ['C' + str(i) for i in cur_clusters]
    return selectedUMAP

## function freq_Matrix(fre_M, cluster_method,para_test)
* **fre_M** is the square matrix recording the number of co-clustering
* **cluster_method** can be 'Hierarchy','Kmeans', 'DBSCAN','HDBSCAN','SNN_community'
* **para_test** is the input parameter dictionary for the cluster method
* **iternum** is the number of iteration to generate the coclustering matrix

In [1]:
import random
import ast
from scipy.spatial.distance import pdist, squareform
import numpy as np
import matplotlib as mpl
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import multiprocessing
import time
def fre_Matrix(fre_M, cluster_method,para_test):
    umapDF = ns.UMAP.iloc[random.sample(range(0,ns.UMAP.shape[0]), int(ns.UMAP.shape[0]*0.95)),:].copy()
    resultDF = get_clusters(umapDF.copy(),method =cluster_method,karg_dict = para_test)
    Crange, Ccounts = np.unique(resultDF["Cluster"], return_counts = True)
    for iter_C in Crange:
        selected_row = resultDF[resultDF["Cluster"]==iter_C]
        Clist = selected_row.index.tolist()
        for sample_row in Clist:
            for sample_col in Clist:
                fre_M.loc[sample_row,sample_col] =  fre_M.loc[sample_row,sample_col]+1
    return fre_M.values
def para_cocluster(cluster_method,para_test,corenum, run_num,ns_input):
    start = time.perf_counter ()
    start=time.time()
    cores = corenum#multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=cores)
    fre_M_t = pd.DataFrame(index = ns_input.UMAP.index, columns =ns_input.UMAP.index)
    fre_M_t [fre_M_t.isnull()]=0
    pool_list=[]
    result_list=[]
    for i in range(run_num):
        pool_list.append(pool.apply_async(fre_Matrix, (fre_M_t, cluster_method, para_test)))
        # 这里不能 get， 会阻塞进程

    #pool.apply_async之后的语句都是阻塞执行的，
    #调用 result.get() 会等待上一个任务执行完之后才会分配下一个任务。
    #事实上，获取返回值的过程最好放在进程池回收之后进行，避免阻塞后面的语句。
    result_list=[xx.get() for xx in pool_list]
    print(sum([xx for xx in  result_list]))
    # 最后我们使用一下语句回收进程池:
    pool.close()
    pool.join()
    elapsed = (time.time() - start)
    print('Time needed to run '+cluster_method + 'is '+ str(elapsed))
    return sum([xx for xx in  result_list])

### For axon morphology features

In [191]:

lm_axon_df_scale= pd.read_excel('/home/penglab/FeaCal/Jmorpho_features/lm_axon_df_scale.xlsx', index_col=0)

index_origin = ns.metadata.index.tolist()


ns.UMAP = nmt.UMAP_wrapper(lm_axon_df_scale, n_neighbors=100, min_dist=0.1, n_components=2, metric='euclidean',PCA_first=True,n_PC=100)
print('Store UMAP for concated Umap')
index_after = ns.UMAP.index.tolist()
index_update = [i for i in index_origin if i in index_after ]
ns.metadata = ns.metadata.loc[index_update,:]
ns.UMAP = ns.UMAP .loc[index_update,:]

Store UMAP for concated Umap


In [ ]:
AM_hier = para_cocluster('Hierarchy', par_hier,30, 5000,ns)
AM_kmeans = para_cocluster('Kmeans', par_kmeans,30, 5000,ns)
AM_dbscan = para_cocluster('DBSCAN', par_dbscan,30, 5000,ns)
AM_hdbscan = para_cocluster('HDBSCAN', par_hdbscan,30, 5000,ns)
AM_snn = para_cocluster('SNN_community', par_snn,30, 5000,ns)

[[4759    0  308 ...    0    0    6]
 [   0 4745   64 ... 4503 4509    0]
 [ 308   64 4725 ...   64   67    0]
 ...
 [   0 4503   64 ... 4750 4517    0]
 [   0 4509   67 ... 4517 4755    0]
 [   6    0    0 ...    0    0 4740]]
Time needed to run Hierarchy is 4603.097372531891
[[4734    0    0 ...    0    0    0]
 [   0 4749    0 ... 4521 4498    0]
 [   0    0 4742 ...    0    0    0]
 ...
 [   0 4521    0 ... 4763 4516    0]
 [   0 4498    0 ... 4516 4738    0]
 [   0    0    0 ...    0    0 4749]]
Time needed to run Hierarchy is 3595.863787651062
[[4727    0 4255 ...    0    0   50]
 [   0 4745    0 ... 4526 4502    0]
 [4255    0 4748 ...    0    0    0]
 ...
 [   0 4526    0 ... 4768 4531    0]
 [   0 4502    0 ... 4531 4746    0]
 [  50    0    0 ...    0    0 4726]]
Time needed to run Hierarchy is 11251.640805006027
[[4764 3556   10 ... 1359    1 4185]
 [3556 4746    9 ... 1971  666 3839]
 [  10    9 4766 ...    3    0   11]
 ...
 [1359 1971    3 ... 4750  895 1467]
 [   1  666 

Exception in thread Thread-17:
Traceback (most recent call last):
  File "/home/penglab/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/home/penglab/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/home/penglab/anaconda3/lib/python3.7/multiprocessing/pool.py", line 412, in _handle_workers
    pool._maintain_pool()
  File "/home/penglab/anaconda3/lib/python3.7/multiprocessing/pool.py", line 248, in _maintain_pool
    self._repopulate_pool()
  File "/home/penglab/anaconda3/lib/python3.7/multiprocessing/pool.py", line 241, in _repopulate_pool
    w.start()
  File "/home/penglab/anaconda3/lib/python3.7/multiprocessing/process.py", line 112, in start
    self._popen = self._Popen(self)
  File "/home/penglab/anaconda3/lib/python3.7/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)
  File "/home/penglab/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", 

In [ ]:
AM_hierDF = pd.DataFrame(data=AM_hier, index=ns.UMAP.index, columns=ns.UMAP.index)
AM_hierDF.to_excel('/home/penglab/FeaCal/dataSource/axonMor/AM_hierDF.xlsx')
AM_kmeansDF = pd.DataFrame(data=AM_kmeans, index=ns.UMAP.index, columns=ns.UMAP.index)
AM_kmeansDF.to_excel('/home/penglab/FeaCal/dataSource/axonMor/AM_kmeansDF.xlsx')
AM_dbscanDF = pd.DataFrame(data=AM_dbscan, index=ns.UMAP.index, columns=ns.UMAP.index)
AM_dbscanDF.to_excel('/home/penglab/FeaCal/dataSource/axonMor/AM_dbscanDF.xlsx')
AM_hdbscanDF = pd.DataFrame(data=AM_hdbscan, index=ns.UMAP.index, columns=ns.UMAP.index)
AM_hdbscanDF.to_excel('/home/penglab/FeaCal/dataSource/axonMor/AM_hdbscanDF.xlsx')
AM_snnDF = pd.DataFrame(data=AM_snn, index=ns.UMAP.index, columns=ns.UMAP.index)
AM_snnDF.to_excel('/home/penglab/FeaCal/dataSource/axonMor/AM_snnDF.xlsx')

### For dendrite morphology features


## Using soma morphology as features
SETTING CLUSTER NUMBER FROM 8 TO 40

### Hierarchy Clustering
For Hierarchy method
* the most suitable parameter is {'L_method': 'weighted', 'L_metric': 'mahalanobis', 'criterionH': 'distance', 'depth': 2, 'R': None, 't': 0.9, 'optimal_ordering': False}
* the ARI is 0.05355799672049869
* The setting cluster number's limit is satisfied, the final number of cluster is 8

### Kmeans Clustering
For Kmeans method
* the most suitable parameter is {'n_clusters': 13, 'init': 'random', 'n_init': 12, 'max_iter': 300, 'tol': 0.0001, 'precompute_distances': True, 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None, 'algorithm': 'auto'}
* the ARI is 0.036079814213255956
* The setting cluster number's limit is satisfied, the final number of cluster is 13

### DBSCAN Clustering
For DBSCAN method
* the most suitable parameter is {'eps': 0.22, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 'leaf_size': 30, 'p': None, 'n_jobs': None}
* the ARI is 0.039530673163787826
* The setting cluster number's limit is satisfied, the final number of cluster is 20

### HDBSCAN Clustering
For HDBSCAN method, 
* the most suitable parameter is {'min_cluster_size': 5, 'metric': 'manhattan', 'alpha': 0.9, 'min_samples': 9, 'p': 2, 'algorithm': 'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 'prediction_data': False, 'match_reference_implementation': False}
* the ARI is 0.020481889417205506
* The setting cluster number's limit is satisfied, the final number of cluster is 21

### SNN Clustering
For SNN_community method
* the most suitable parameter is {'knn': 5, 'metric': 'minkowski', 'method': 'FastGreedy'}
* the ARI is 0.045224072138369056
* The setting cluster number's limit is satisfied, the final number of cluster is 18

In [ ]:
lm_dendrite_df_scale= pd.read_excel('/home/penglab/FeaCal/Jmorpho_features/lm_dendrite_df_scale.xlsx', index_col=0)
index_origin = ns.metadata.index.tolist()
ns.UMAP = nmt.UMAP_wrapper(lm_dendrite_df_scale, n_neighbors=100, min_dist=0.1, n_components=2, metric='euclidean',PCA_first=True,n_PC=100)
print('Store UMAP for concated Umap')

index_after = ns.UMAP.index.tolist()
index_update = [i for i in index_origin if i in index_after ]
ns.metadata = ns.metadata.loc[index_update,:]
ns.UMAP = ns.UMAP .loc[index_update,:]

In [ ]:
par_hier2 =  {'L_method': 'weighted', 'L_metric': 'mahalanobis', 'criterionH': 'distance', 'depth': 2, 'R': None, 
              't': 0.9, 'optimal_ordering': False}
par_kmeans2 = {'n_clusters': 13, 'init': 'random', 'n_init': 12, 'max_iter': 300, 'tol': 0.0001,
               'precompute_distances': True, 'verbose': 0, 'random_state': None, 'copy_x': True, 'n_jobs': None,
               'algorithm': 'auto'}

par_dbscan2 = {'eps': 0.22, 'min_samples': 5, 'metric': 'euclidean', 'metric_params': None, 'algorithm': 'auto', 
               'leaf_size': 30, 'p': None, 'n_jobs': None}

par_hdbscan2 =  {'min_cluster_size': 5, 'metric': 'manhattan', 'alpha': 0.9, 'min_samples': 9, 'p': 2, 'algorithm': 'generic', 'leaf_size': 40, 'approx_min_span_tree': True, 'gen_min_span_tree': False, 'core_dist_n_jobs': 4, 'cluster_selection_method': 'eom', 'allow_single_cluster': False, 'prediction_data': False, 'match_reference_implementation': False}

par_snn2 = {'knn':5,'metric':'minkowski','method':'FastGreedy'}


In [ ]:
DM_hier = para_cocluster('Hierarchy', par_hier2,30, 5000,ns)
DM_kmeans = para_cocluster('Kmeans', par_kmeans2,30, 5000,ns)
DM_dbscan = para_cocluster('DBSCAN', par_dbscan2,30, 5000,ns)
DM_hdbscan = para_cocluster('HDBSCAN', par_hdbscan2,30, 5000,ns)
DM_snn = para_cocluster('SNN_community', par_snn2,30, 5000,ns)

In [ ]:
DM_hierDF = pd.DataFrame(data=DM_hier, index=ns.UMAP.index, columns=ns.UMAP.index)
DM_hierDF.to_excel('/home/penglab/FeaCal/dataSource/denMor/DM_hierDF.xlsx')
DM_kmeansDF = pd.DataFrame(data=DM_kmeans, index=ns.UMAP.index, columns=ns.UMAP.index)
DM_kmeansDF.to_excel('/home/penglab/FeaCal/dataSource/denMor/DM_kmeansDF.xlsx')
DM_dbscanDF = pd.DataFrame(data=DM_dbscan, index=ns.UMAP.index, columns=ns.UMAP.index)
DM_dbscanDF.to_excel('/home/penglab/FeaCal/dataSource/denMor/DM_dbscanDF.xlsx')
DM_hdbscanDF = pd.DataFrame(data=DM_hdbscan, index=ns.UMAP.index, columns=ns.UMAP.index)
DM_hdbscanDF.to_excel('/home/penglab/FeaCal/dataSource/denMor/DM_hdbscanDF.xlsx')
DM_snnDF = pd.DataFrame(data=DM_snn, index=ns.UMAP.index, columns=ns.UMAP.index)
DM_snnDF.to_excel('/home/penglab/FeaCal/dataSource/denMor/DM_snnDF.xlsx')